# Collisions 

In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
dtype_collisions={
    'case_id': str, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'collision_date': str, #datetime.date,
    'collision_severity': str,# => convert to ID
    'collision_time': str, #time, 
    'county_city_location': str,
    'hit_and_run': str,# => convert to ID
    'jurisdiction': pd.Int16Dtype(),
    'lighting': str,
    'location_type': str,
    'officer_id': str,
    'pcf_violation': pd.Int64Dtype(),
    'pcf_violation_category': str,
    'pcf_violation_subsection': str,
    'population': pd.Int64Dtype(),
    'primary_collision_factor': str,
    'process_date': str, #datetime.date,
    'ramp_intersection': pd.Int64Dtype(),
    'road_condition_1': str,
    'road_condition_2': str,
    'road_surface': str,
    'tow_away': 'boolean',
    'type_of_collision': str,
    'weather_1': str,
    'weather_2': str, 
}

In [3]:
collisions_data = pd.read_csv("CSV-2018/collisions2018.csv", dtype=dtype_collisions ,usecols=dtype_collisions.keys()) #

In [4]:
collisions_data

case_id collision_date    collision_severity  \
0                    0000001     2002-01-18  property damage only   
1                    0000002     2002-02-13  property damage only   
2                    0000003     2002-02-11  property damage only   
3                    0000004     2002-02-12  property damage only   
4                    0000005     2002-02-14         severe injury   
...                      ...            ...                   ...   
3678058  9870011224092016011     2001-12-24                  pain   
3678059  9870011224123011850     2001-12-24  property damage only   
3678060  9870011226102009803     2001-12-26          other injury   
3678061  9870011228210011458     2001-12-28  property damage only   
3678062  9870011231152508671     2001-12-31                  pain   

        collision_time county_city_location      hit_and_run  jurisdiction  \
0             15:30:00                 1900  not hit and run          9575   
1             19:30:00                 1500      misdemeanor          9420   
2             15:30:00                 1502      misdemeanor          9420   
3             07:45:00                 1500      misdemeanor          9420   
4             11:35:00                 1500  not hit and run          9420   
...                ...                  ...              ...           ...   
3678058       09:20:00                 3600  not hit and run          9870   
3678059       12:30:00                 3600  not hit and run          9870   
3678060       10:20:00                 3600  not hit and run          9870   
3678061       21:00:00                 3600      misdemeanor          9870   
3678062       15:25:00                 3600  not hit and run          9870   

                           lighting location_type officer_id  ...  \
0                          daylight           NaN      16418  ...   
1                               NaN           NaN      14360  ...   
2                          daylight       highway      16117  ...   
3                          daylight           NaN       6078  ...   
4                          daylight           NaN      13851  ...   
...                             ...           ...        ...  ...   
3678058                    daylight       highway      16011  ...   
3678059                    daylight  intersection      11850  ...   
3678060                    daylight           NaN       9803  ...   
3678061  dark with no street lights           NaN      11458  ...   
3678062                    daylight       highway       8671  ...   

         primary_collision_factor process_date ramp_intersection  \
0          vehicle code violation   2002-06-14              <NA>   
1          vehicle code violation   2002-03-29              <NA>   
2          vehicle code violation   2003-07-15              <NA>   
3          vehicle code violation   2002-05-22              <NA>   
4          vehicle code violation   2003-07-28              <NA>   
...                           ...          ...               ...   
3678058    vehicle code violation   2002-04-26                 7   
3678059    vehicle code violation   2002-04-26                 5   
3678060    vehicle code violation   2002-03-29                 8   
3678061    vehicle code violation   2002-03-08                 8   
3678062    vehicle code violation   2002-04-26                 7   

         road_condition_1 road_condition_2 road_surface  tow_away  \
0                  normal              NaN          dry      True   
1                  normal              NaN          dry     False   
2                  normal              NaN          dry     False   
3                  normal              NaN          dry     False   
4                  normal              NaN          dry      True   
...                   ...              ...          ...       ...   
3678058            normal              NaN          dry      True   
3678059            normal              NaN          dry  

# Data Cleaning

Translation tables

In [5]:
translation_collision_severity ={
    "fatal": 1,
    "severe injury": 2,
    "other injury": 3,
    "pain": 4,
    "property damage only": 0,
}

translation_hit_and_run ={
    "felony": "F",
    "misdemeanor": "M",
    "not hit and run": "N",
}

translation_lighting ={
    "daylight": "A",
    "dusk or dawn": "B",
    "dark with street lights": "C",
    "dark with no street lights": "D",
    "dark with street lights not functioning": "E",
}

translation_location_type ={
    "highway": "H",
    "intersection": "I",
    "ramp": "R",
}

translation_pcf_violation_category ={
    'unknown': 0,
    'dui': 1,
    'impeding traffic': 2,
    'speeding': 3,
    'following too closely': 4,
    'wrong side of road': 5,
    'improper passing': 6,
    'unsafe lane change': 7,
    'improper turning': 8,
    'automobile right of way':9,
    'pedestrian right of way': 10,
    'pedestrian violation': 11,
    'traffic signals and signs': 12,
    'hazardous parking': 13,
    'lights': 14,
    'brakes': 15,
    'other equipment': 16,
    'other hazardous violation': 17,
    'other than driver (or pedestrian)': 18,
    'unsafe starting or backing': 19,
    'other improper driving': 22,
    'pedestrian dui': 23,
    'fell asleep': 24,    
}

translation_primary_collision_factor ={
    "vehicle code violation": "A",
    "other improper driving": "B",
    "other than driver": "C",
    "unknown": "D",
    "fell asleep": "E",
}

translation_road_condition ={
    'holes': "A",
    'loose material': "B",
    'obstruction': "C",
    'construction': "D",
    'reduced width':"E",
    'flooded': "F",
    'other':"G",
    'normal': "H",
}

translation_road_surface ={
    'dry': "A",
    'wet': "B",
    'snowy': "C", 
    'slippery': "D", 
}

translation_type_of_collision ={
    'head-on': "A",
    'sideswipe':"B",
    'rear end':"C",
    'broadside':"D",
    'hit object':"E",
    'overturned':"F",
    'pedestrian':"G",
    'other':"H",
}

translation_weather ={
    'clear': "A", 
    'cloudy': "B",
    'raining': "C",
    'snowing':"D",
    'fog': "E",
    'other': "F", 
    'wind':"G",
}

Clean columns that need to be cleaned

In [6]:
clean_data = collisions_data.copy()
clean_data["collision_severity"] = collisions_data["collision_severity"].map(lambda s: translation_collision_severity[s]).astype(pd.Int16Dtype())
clean_data["hit_and_run"] = collisions_data["hit_and_run"].map(lambda s: translation_hit_and_run.get(s, None))
clean_data["lighting"] = collisions_data["lighting"].map(lambda s: translation_lighting.get(s, None))
clean_data["location_type"] = collisions_data["location_type"].map(lambda s: translation_location_type.get(s, None))
clean_data["primary_collision_factor"] = collisions_data["primary_collision_factor"].map(lambda s: translation_primary_collision_factor.get(s, None))
clean_data["pcf_violation_category"] = collisions_data["pcf_violation_category"].map(lambda s: translation_pcf_violation_category.get(s, None)).astype(pd.Int16Dtype())
clean_data["road_condition_1"] = collisions_data["road_condition_1"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_condition_2"] = collisions_data["road_condition_2"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_surface"] = collisions_data["road_surface"].map(lambda s: translation_road_surface.get(s, None))
clean_data["type_of_collision"] = collisions_data["type_of_collision"].map(lambda s: translation_type_of_collision.get(s, None))
clean_data["weather_1"] = collisions_data["weather_1"].map(lambda s: translation_weather.get(s, None))
clean_data["weather_2"] = collisions_data["weather_2"].map(lambda s: translation_weather.get(s, None))
clean_data["process_date"] = collisions_data["process_date"].astype("datetime64[ns]").dt.date
clean_data["collision_date"] = collisions_data["collision_date"].astype("datetime64[ns]")
clean_data["collision_time"] = ("2000-01-01 "+ collisions_data["collision_time"]).astype("datetime64[ns]")

In [7]:
clean_data

case_id collision_date  collision_severity  \
0                    0000001     2002-01-18                   0   
1                    0000002     2002-02-13                   0   
2                    0000003     2002-02-11                   0   
3                    0000004     2002-02-12                   0   
4                    0000005     2002-02-14                   2   
...                      ...            ...                 ...   
3678058  9870011224092016011     2001-12-24                   4   
3678059  9870011224123011850     2001-12-24                   0   
3678060  9870011226102009803     2001-12-26                   3   
3678061  9870011228210011458     2001-12-28                   0   
3678062  9870011231152508671     2001-12-31                   4   

             collision_time county_city_location hit_and_run  jurisdiction  \
0       2000-01-01 15:30:00                 1900           N          9575   
1       2000-01-01 19:30:00                 1500           M          9420   
2       2000-01-01 15:30:00                 1502           M          9420   
3       2000-01-01 07:45:00                 1500           M          9420   
4       2000-01-01 11:35:00                 1500           N          9420   
...                     ...                  ...         ...           ...   
3678058 2000-01-01 09:20:00                 3600           N          9870   
3678059 2000-01-01 12:30:00                 3600           N          9870   
3678060 2000-01-01 10:20:00                 3600           N          9870   
3678061 2000-01-01 21:00:00                 3600           M          9870   
3678062 2000-01-01 15:25:00                 3600           N          9870   

        lighting location_type officer_id  ...  primary_collision_factor  \
0              A          None      16418  ...                         A   
1           None          None      14360  ...                         A   
2              A             H      16117  ...                         A   
3              A          None       6078  ...                         A   
4              A          None      13851  ...                         A   
...          ...           ...        ...  ...                       ...   
3678058        A             H      16011  ...                         A   
3678059        A             I      11850  ...                         A   
3678060        A          None       9803  ...                         A   
3678061        D          None      11458  ...                         A   
3678062        A             H       8671  ...                         A   

         process_date ramp_intersection  road_condition_1 road_condition_2  \
0          2002-06-14              <NA>                 H             None   
1          2002-03-29              <NA>                 H             None   
2          2003-07-15              <NA>                 H             None   
3          2002-05-22              <NA>                 H             None   
4          2003-07-28              <NA>                 H             None   
...               ...               ...               ...              ...   
3678058    2002-04-26                 7                 H             None   
3678059    2002-04-26                 5                 H             None   
3678060    2002-03-29                 8                 H             None   
3678061    2002-03-08                 8                 H             None   
3678062    2002-04-26                 7                 H             None   

        road_surface  tow_away type_of_collision weather_1 weather_2  
0                  A      True                 C         A      None  
1                  A     False                 E         A      None  
2                  A     False                 E         A      None  
3                  A     False                 B         A      None  
4                  A      True                 F         A      None  
...

                                ------------------------------------------

# Create CSV files

## Collisions CSV

In [8]:
collisions_csv = clean_data.copy()

collision_rename_translate = {"collision_severity": "collision_severity_id", "hit_and_run": "hit_and_run_id", "lighting": "lighting_id", "location_type": "location_type_id", "pcf_violation_category": "pcf_violation_category_id", "population":"population_id", "primary_collision_factor": "primary_collision_factor_id", "ramp_intersection":"ramp_intersection_id", "road_surface":"road_surface_id", "type_of_collision": "type_of_collision_id"}
collision_drop = ["road_condition_1", "road_condition_2", "weather_1", "weather_2"]
collisions_csv.rename(columns=collision_rename_translate, inplace=True)
collisions_csv.drop(collision_drop , axis="columns",  inplace=True)
collisions_csv.loc[collisions_csv['case_id']=='2820565','officer_id'] = None

We decided to change the officer id ",66" to None because we had problems inserting it in the database due to the ','. We could have changed it to "66" (which is a valid value in the dataset), but since we were not sure that it was a typo, we found this assumption too strong and therefore we prefered to remove it.

In [9]:
collisions_csv.to_csv('out/collision.csv', index=False)

## with_weather relation CVS

In [10]:
translate= {"weather_1": "wheather_id", "weather_2": "wheather_id"}
weather_1 = clean_data[~clean_data["weather_1"].isnull()][["case_id", "weather_1"]].copy().rename(columns=translate)
weather_2 = clean_data[~clean_data["weather_2"].isnull()][["case_id", "weather_2"]].copy().rename(columns=translate)
with_weather_csv = pd.concat([weather_1, weather_2], axis=0).drop_duplicates()

In [11]:
with_weather_csv.to_csv('out/collision_with_weather.csv', index=False)

## with_road_condition relation CVS

In [12]:
translate= {"road_condition_1": "road_condition_id", "road_condition_2": "road_condition_id"}
road_condition_1 = clean_data[~clean_data["road_condition_1"].isnull()][["case_id", "road_condition_1"]].copy().rename(columns=translate)
road_condition_2 = clean_data[~clean_data["road_condition_2"].isnull()][["case_id", "road_condition_2"]].copy().rename(columns=translate)
with_road_condition_csv = pd.concat([road_condition_1, road_condition_2], axis=0).drop_duplicates()

In [13]:
with_road_condition_csv.to_csv('out/collision_with_road_condition.csv', index=False)